In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.0 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import json
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [ ]:
url_jac = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/atencion-ciudadana/arbolado-publico-lineal/arbolado-publico-lineal-2017-2018.csv"

In [ ]:
jac = pd.read_csv(url_jac)

<ipython-input-4-9543c713645c>:1: DtypeWarning: Columns (2,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  jac = pd.read_csv(url_jac)


In [ ]:
url_barrios = urlopen('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson')

In [ ]:
def coordinates_extract(geojson, empty_list, empty_dict):
  for feature in geojson['features']:
    if feature['geometry']['type'] == "MultiPolygon":
      for polyg in feature['geometry']['coordinates']:
        empty_list.extend(polyg[0])
        empty_list.append([None, None])

In [ ]:
barrios = json.load(url_barrios)

barrios_coords = []

coordinates_extract(barrios, barrios_coords, None)

barrios_lons, barrios_lats = zip(*barrios_coords)

In [ ]:
barrios_CABA = go.Scattergeo(
    lat=barrios_lats,
    lon=barrios_lons,
    mode='lines',
    line=dict(color='white', width=1),
    opacity=0.5,
    showlegend=False,
    hoverinfo='skip'
)

In [ ]:
jacarandas = go.Scattergeo(
    lat=jac[jac["nombre_cientifico"] == 'Jacaranda mimosifolia']["lat"],
    lon=jac[jac["nombre_cientifico"] == 'Jacaranda mimosifolia']["long"],
    mode='markers',
    marker=dict(
        color='#848DC5',
        size=2
    ),
    text=jac[jac["nombre_cientifico"] == 'Jacaranda mimosifolia']["direccion_normalizada"],
    hoverinfo="text",
    opacity=1,
    showlegend=False
)

In [ ]:
def maps_layout_and_annotation(fig):
  fig.update_layout(
      paper_bgcolor='#28192c',
      margin=dict(
          t = 150,
          b = 0,
          l = 0,
          r = 0
      ),
      height = 900,
      width = 900,
      title = dict(
        text = "<b>Jacarandás en Buenos Aires</b><br><sup>Ubicación geográfica de jacarandás en CABA (2017-2018).<br><sup><b>Fuente</b>: BA Data.</sup></sup>",
        xanchor = 'left',
        x = 0.0325,
        yanchor = 'top',
        y = 0.95,
        font = dict(
            size = 26,
            family="Courier New",
            color = 'white'
        )
        ),
      geo = dict(
        center = dict(
            lat = -34.616,
            lon = -58.435
        ),
        bgcolor = '#28192c',
        projection_type = 'mercator',
        projection_scale = 1640,
        resolution = 50,
        showlakes = False,
        showcoastlines = False,
        showland = False,
        showocean = False,
        showframe = False,
        showcountries = False,
    )
  )

In [ ]:
mapa_jac = go.Figure()

In [ ]:
mapa_jac.add_trace(barrios_CABA)

mapa_jac.add_trace(jacarandas)

maps_layout_and_annotation(mapa_jac)

In [ ]:
mapa_jac

In [ ]:
pio.write_html(mapa_jac,
               file='mapa_jac.html',
               full_html=True)